In [9]:
import os
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from tqdm import tqdm

def fetch_product_urls(pet_type):
    product_urls = []
    
    with webdriver.Chrome() as driver:
        if pet_type == 'dog':
            total_pages = 10900
            base_url = 'https://search.shopping.naver.com/search/all?adQuery=%EA%B0%95%EC%95%84%EC%A7%80%20%EC%82%AC%EB%A3%8C&frm=NVSHATC&npayType=2&origQuery=%EA%B0%95%EC%95%84%EC%A7%80%20%EC%82%AC%EB%A3%8C&pagingIndex={}&pagingSize=40&productSet=checkout&query=%EA%B0%95%EC%95%84%EC%A7%80%20%EC%82%AC%EB%A3%8C&sort=rel&timestamp=&viewType=image'
        elif pet_type == 'cat':
            total_pages = 5900
            base_url = 'https://search.shopping.naver.com/search/all?adQuery=%EA%B3%A0%EC%96%91%EC%9D%B4%20%EC%82%AC%EB%A3%8C&frm=NVSHATC&npayType=2&origQuery=%EA%B3%A0%EC%96%91%EC%9D%B4%20%EC%82%AC%EB%A3%8C&pagingIndex={}&pagingSize=40&productSet=checkout&query=%EA%B3%A0%EC%96%91%EC%9D%B4%20%EC%82%AC%EB%A3%8C&sort=rel&timestamp=&viewType=image'
        
        for page in tqdm(range(1, total_pages+1)[:10], desc='Fetching product URLs'):
            retry_count = 3
            while retry_count > 0:
                try:
                    url = base_url.format(page)
                    driver.get(url)
                    driver.implicitly_wait(2)

                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    driver.implicitly_wait(2)

                    links = driver.find_elements(By.CSS_SELECTOR, '.thumbnail_thumb__Bxb6Z.linkAnchor')

                    for link in links:
                        product_urls.append(link.get_attribute('href'))
                    break
                except Exception as e:
                    print(f'Error: {e}')
                    retry_count -= 1
                    driver.refresh()
                    print('Retrying...')
    
    if product_urls:
        with open(f'{pet_type}_product_urls.txt', 'w') as file:
            for url in product_urls:
                file.write(url + '\n')

In [14]:
def download_images(urls, pet_type):
    with webdriver.Chrome() as driver:
        for url in urls:
            retry_count = 1
            while retry_count > 0:
                try:
                    driver.get(url)
                    
                    driver.find_element(By.CSS_SELECTOR, '#INTRODUCE > div > div._3osy73V_eD._1Hc_ju_IXp > button').click()
                    
                    product_name = driver.find_elements(By.CSS_SELECTOR, '._1eddO7u4UC > h3').text
                    
                    main_container = driver.find_elements(By.CSS_SELECTOR,'.se-main-container')

                    image_elements = main_container.find_elements(By.TAG_NAME, 'img')
                    image_urls = [img.get_attribute('src') for img in image_elements]
                    
                    path = f'naver/{pet_type}/{product_name}'
                        
                    if not os.path.exists(path):
                        os.makedirs(path)

                    for index, img_url in enumerate(image_urls):
                        img_data = requests.get(img_url).content
                        with open(f'{path}/image_{index + 1}.jpg', 'wb') as handler:
                            handler.write(img_data)
                    break
                except Exception as e:
                    print(f'Error: {e}')
                    retry_count -= 1
                    driver.refresh()
                    print('Retrying...')

In [10]:
pet_type = 'dog'
# pet_type = 'cat'

fetch_product_urls(pet_type)

Fetching product URLs: 100%|██████████| 10/10 [00:23<00:00,  2.37s/it]


In [15]:
file_path = 'dog_product_urls.txt'
# file_path = 'cat_product_urls.txt'

with open(file_path, 'r') as file:
    urls = file.readlines()
    
download_images(urls, pet_type)

Error: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#INTRODUCE > div > div._3osy73V_eD._1Hc_ju_IXp > button"}
  (Session info: chrome=121.0.6167.86); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF74BC95E42+3538674]
	(No symbol) [0x00007FF74B8B4C02]
	(No symbol) [0x00007FF74B765AEB]
	(No symbol) [0x00007FF74B7ABF4E]
	(No symbol) [0x00007FF74B7AC0CC]
	(No symbol) [0x00007FF74B7EE477]
	(No symbol) [0x00007FF74B7CF0EF]
	(No symbol) [0x00007FF74B7EBDE3]
	(No symbol) [0x00007FF74B7CEE53]
	(No symbol) [0x00007FF74B79F514]
	(No symbol) [0x00007FF74B7A0631]
	GetHandleVerifier [0x00007FF74BCC6CAD+3738973]
	GetHandleVerifier [0x00007FF74BD1C506+4089270]
	GetHandleVerifier [0x00007FF74BD14823+4057299]
	GetHandleVerifier [0x00007FF74B9E5C49+720121]
	(No symbol) [0x00007FF74B8C126F]
	(No symbol) [0x00007FF74B8BC304]
	(

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=121.0.6167.86)
Stacktrace:
	GetHandleVerifier [0x00007FF74BC95E42+3538674]
	(No symbol) [0x00007FF74B8B4C02]
	(No symbol) [0x00007FF74B765AEB]
	(No symbol) [0x00007FF74B74288C]
	(No symbol) [0x00007FF74B7D5DD7]
	(No symbol) [0x00007FF74B7EB40F]
	(No symbol) [0x00007FF74B7CEE53]
	(No symbol) [0x00007FF74B79F514]
	(No symbol) [0x00007FF74B7A0631]
	GetHandleVerifier [0x00007FF74BCC6CAD+3738973]
	GetHandleVerifier [0x00007FF74BD1C506+4089270]
	GetHandleVerifier [0x00007FF74BD14823+4057299]
	GetHandleVerifier [0x00007FF74B9E5C49+720121]
	(No symbol) [0x00007FF74B8C126F]
	(No symbol) [0x00007FF74B8BC304]
	(No symbol) [0x00007FF74B8BC432]
	(No symbol) [0x00007FF74B8ABD04]
	BaseThreadInitThunk [0x00007FFFDFE8257D+29]
	RtlUserThreadStart [0x00007FFFE0ACAA58+40]
